In [41]:
import os

In [45]:
cd abdulrasheed/Desktop/ImageClassification

/Users/abdulrasheed/Desktop/ImageClassification


In [46]:
%pwd

'/Users/abdulrasheed/Desktop/ImageClassification'

In [79]:
# srouce/CNNClassifier/entity/config_entity.py

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir : Path
    source_URL : str
    local_data_file : Path
    unzip_dir : Path

In [80]:
# source/CNNClassfier/config/configuration.py

from source.CNNClassifier.constant.__init_ import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from source.CNNClassifier.utils.common import read_yaml, create_directories

In [81]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH, # stores cofig file path 'config/config.yaml'
        params_filepath = PARAMS_FILE_PATH): # stores params file path 'params.yaml'

        self.config = read_yaml(config_filepath) # reads the config.yaml file
        self.params = read_yaml(params_filepath) # reads the params.yaml file

        create_directories([self.config.artifacts_root]) # creates artifacts folder 'artifacts_root : artifacts'

    def get_data_ingestion_config(self) -> DataIngestionConfig:

        config = self.config.data_ingestion # config stores artifacts/data_ingestion
        # Accesses the data_ingestion section from the loaded configuration (self.config)
        # This section comes from a YAML file (parsed earlier by the read_yaml function) and contains all settings related to data ingestion.


        create_directories([config.root_dir]) # creates data_ingestion folder (root_dir : artifacts/data_ingestion)

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,              # artifacts/data_ingestion
            source_URL = config.source_URL,          #'https://github.com/AbdulRasheed1011/ImagesData/blob/main/Images.zip'
            local_data_file= config.local_data_file, # artifacts/data_ingestion/data.zip
            unzip_dir = config.unzip_dir             # artifacts/data_ingestion
        )

        return data_ingestion_config             


In [82]:
### Components data_ingestion.py

In [83]:
import os
import urllib.request as request
import zipfile  
from source.CNNClassifier import logger
from source.CNNClassifier.utils.common import get_size

In [84]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config 

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, header = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file

            )
            logger.info(f"{filename} download! with following info: \n{header}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        """
        zip_file_path : str
        extract zip file to data diretory
        Function returns None
        """

        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok = True)
        print(f"File path: {self.config.local_data_file}")

        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
            logger.info(f"Extracted zip file to {unzip_path}") 


In [85]:
### pipeline

In [86]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config = data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e
    logger.error(f"Error during data ingestion: {str(e)}")

[2024-12-09 20:51:16,183: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-12-09 20:51:16,186: INFO: common: yaml file: params.yaml loaded successfully]
[2024-12-09 20:51:16,186: INFO: common: created directory at: artifacts]
[2024-12-09 20:51:16,187: INFO: common: created directory at: artifacts/data_ingestion]
[2024-12-09 20:51:17,689: INFO: 2873370200: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 67446398
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "d5f5b4e21b3db22d19965753fd565933be473e744a85accaed9a02fc0a4925d4"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 268C:348C2A:1A3737:1D5388:6757ABD8
Accept-Ranges: bytes
Date: Tue, 10 Dec 2024 02:51:16 GMT
Via: 1.1 varnish
X-Served-By: cache-dfw-ktki8620040-DFW
X-

In [88]:
import os
from typing import List

def get_folder_names(directory_path: str) -> List[str]:
    """
    Get all folder names in the given directory.

    Parameters:
        directory_path (str): Path to the directory.

    Returns:
        List[str]: List of folder names in the directory.
    """
    if not os.path.exists(directory_path):
        raise FileNotFoundError(f"Directory {directory_path} does not exist.")
    
    folder_names = [
        entry for entry in os.listdir(directory_path) 
        if os.path.isdir(os.path.join(directory_path, entry))
    ]
    return folder_names

if __name__ == "__main__":
    # Define the path to the directory
    images_dir = "artifacts/data_ingestion/Images"
    
    try:
        # Get folder names
        names_in_images = get_folder_names(images_dir)
        print("Folder names in 'Images':", names_in_images)
    except Exception as e:
        print(f"An error occurred: {e}")



Folder names in 'Images': ['cat', 'dog', 'men', 'women']
